#  **Identificando discurso de ódio pelo Qwen 3**

**Autores:** Luiz Fernando de Aguiar Lima, Hérique Costa Ribeiro de Lima, Adonias Caetano de Oliveira

## **Installation**

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
!pip install Unidecode

## **Dataset**

In [ ]:
import pandas as pd
import re
import random
import numpy as np
import torch

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1lOKAxCtQ7v0q0cL_IfxhLDKCMbDtbV6B/edit?usp=sharing&ouid=107874559379957908991&rtpof=true&sd=true'

file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id

# read the data
dataset_hate = pd.read_excel(read_url, index_col=0)

dataset_hate['class'] = dataset_hate['class'].astype(int)

# display the first 5 rows
dataset_hate

,tweet,class
0,"@jennybaquing so terms like honkey, cracker, p...",0
1,I think dat @Miss_Facetious called me a nigger...,0
2,RT @UNCLERUCKUS_BD: Being a nigger isn't a col...,0
3,"@NYRKelsMads @DaniloNavarro85 Hey fag, I said ...",0
4,RT @DefendWallSt: ...wow? RT @1SleepingGiant: ...,0
...,...,...
145,Wait did @kidfivevibes take my brownies??,2
146,Parodies of Jihadi songs - more effective than...,2
147,Just stopped by the line up room @BeaLack stil...,2
148,Lauren &amp; I ate candy we found on the floor...,2


## **Unsloth**

In [ ]:
from unsloth import FastLanguageModel


fourbit_models = [
    "unsloth/Qwen3-1.7B-unsloth-bnb-4bit", # Qwen 14B 2x faster
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

==((====))==  Unsloth 2025.7.8: Fast Qwen3 patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

## **Experimentos de identificação de ideação suicida**





<a name="Inference"></a>
### **Inference**
De acordo com a equipe `Qwen-3`, as configurações recomendadas para inferência de raciocínio são `temperature = 0.6, top_p = 0.95, top_k = 20`

Para inferência baseada em bate-papo normal, `temperature = 0.7, top_p = 0.8, top_k = 20`

In [ ]:
respostas_llm = {}

sentencas = dataset_hate["tweet"].to_list()
targets = dataset_hate["class"].to_list()

respostas_llm["text"] = sentencas
respostas_llm["target"] = targets
respostas_llm['predicted'] = []

for sentenca in sentencas:

  prompt = f"""
    Você é um modelo de IA treinado para classificar frases como:

    - "0" se a frase contém discurso de ódio (ataques direcionados a grupos com base em raça, religião, orientação sexual, etc.).
    - "1" se a frase contém linguagem ofensiva, mas não for discurso de ódio.
    - "2" se a frase não contém discurso de ódio nem linguagem ofensiva

    Regras:
    - Responda *apenas* com "0", "1" ou "2".
    - Não adicione explicações ou qualquer outro texto.

    Frase: "{sentenca}"
    """

  messages = [
      {"role" : "user", "content" :  prompt}
  ]
  text = tokenizer.apply_chat_template(
      messages,
      tokenize = False,
      add_generation_prompt = True, # Must add for generation
      enable_thinking = True, # Disable thinking
  )

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference

  inputs = tokenizer(text, return_tensors = "pt").to("cuda")
  output = model.generate(**inputs, max_new_tokens=1024, temperature = 0.6, top_p = 0.95, top_k = 20)
  resposta_ids = output[0]
  resposta_texto = tokenizer.decode(resposta_ids, skip_special_tokens=True)
  resposta_texto

  #respostas_llm['predicted'].append(resposta_texto.split("</think>")[1].strip())
  respostas_llm['predicted'].append(resposta_texto.split("texto.")[1].strip())


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:

from google.colab import files

df = pd.DataFrame(respostas_llm)
df.to_excel('respostas_qwen3.xlsx', index=False)

files.download('respostas_qwen3.xlsx')
